# Analyse de la Répartition des Élus par Département et par Sexe en France


Le contexte de cette analyse est la compréhension de la répartition géographique et démographique des élus en France. Le fichier source utilisé provient du Répertoire National des Élus, disponible sur le site data.gouv.fr à l'adresse https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv. Ce répertoire contient des informations détaillées sur les élus, notamment leurs données personnelles, professionnelles et les détails de leurs mandats.


L'objectif de cette analyse est d'explorer la répartition des élus par département et par sexe, afin de mettre en évidence les disparités géographiques et de genre parmi les élus français. Cette analyse permettra de comprendre comment les élus sont répartis à travers les différents départements de France et d'identifier les tendances en matière de représentation des femmes et des hommes dans la sphère politique locale.


## Méthodologie


La méthodologie utilisée pour cette analyse implique plusieurs étapes. Tout d'abord, les données sont chargées et traitées à l'aide de requêtes SQL pour extraire les informations nécessaires. Une requête spécifique a été générée pour compter le nombre d'élus par département et par sexe. Les données résultantes sont ensuite traitées et transformées pour être utilisées dans une visualisation géographique interactive. Cette visualisation utilise une carte de France divisée par départements, où chaque département est coloré en fonction du ratio de femmes élues. La visualisation est créée à l'aide de la bibliothèque Plotly Express et des données géographiques provenant d'un fichier GeoJSON.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
 SUBSTR("Code du département", 1, 2) AS code_dept,
 "Code sexe" AS sexe,
 COUNT(*) AS nb_elus
FROM 
 loaded_dataset
GROUP BY 
 1, "Code sexe"
ORDER BY 
 code_dept, sexe """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui est adaptée pour créer des visualisations interactives et attractives. Dans ce cas, elle est utilisée pour créer une carte choropleth interactive qui représente le ratio de femmes élues par département en France. Cela permet une exploration facile et intuitive des données géographiques.

In [3]:
import pandas as pd
import duckdb as ddb
import plotly.express as px
import geopandas as gpd
import pandas as pd
import requests

# Récupération du GeoJSON des départements français
url_geo = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
gdf_depts = gpd.read_file(url_geo)

# Préparation des données : pivot pour avoir le nombre d'élus par sexe par département
df_pivot = df.pivot(index='code_dept', columns='sexe', values='nb_elus').reset_index()
df_pivot['ratio_femmes_pct'] = (df_pivot['F'] / (df_pivot['F'] + df_pivot['M']) * 100).round(1)
df_pivot['nb_elus_total'] = df_pivot['F'] + df_pivot['M']

# Fusion avec les données géographiques
gdf_depts['code'] = gdf_depts['code'].astype(str).str.zfill(2)
df_pivot['code_dept'] = df_pivot['code_dept'].astype(str).str.zfill(2)
gdf_merged = gdf_depts.merge(df_pivot, left_on='code', right_on='code_dept', how='inner')

# Création de la carte interactive
dataviz = px.choropleth_mapbox(
    gdf_merged,
    geojson=gdf_merged.geometry.__geo_interface__,
    locations=gdf_merged.index,
    color='ratio_femmes_pct',
    color_continuous_scale='RdBu',
    range_color=[gdf_merged['ratio_femmes_pct'].min(), gdf_merged['ratio_femmes_pct'].max()],
    mapbox_style="carto-positron",
    zoom=4.5,
    center={"lat": 46.5, "lon": 2.0},
    opacity=0.7,
    labels={'ratio_femmes_pct': 'Ratio femmes (%)'},
    hover_data={
        'nom': True,
        'F': ':.0f',
        'M': ':.0f',
        'nb_elus_total': ':.0f',
        'ratio_femmes_pct': ':.1f%'
    },
    title="Répartition des élus par département et par sexe - Ratio de femmes élues (%)"
)

dataviz.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    font_family="Arial",
    title_font_size=14,
    title_x=0.5
)

# Inversion de l'échelle de couleurs pour mettre en évidence les femmes
dataviz.update_traces(
    colorbar=dict(
        title='Ratio<br>femmes élues (%)',
        ticksuffix='%',
        len=0.7,
        yanchor='middle',
        y=0.5
    )
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*